In [ ]:
# Importing libraries
import pandas as pd
import os
from datetime import datetime
import shutil
from pathlib import Path

## NEED TO map correct names for A FEW OF THE ORGANIZATIONS LIKE IN THE FILE 12/8/2025

In [ ]:
# Where the files will be loaded from 

# change the location below to make sure that the file path is correct for your computer
onedrive_base = Path(os.environ.get('onedrive' , 'C:/Users/azona/OneDrive - Cerity Partners'))
process_base = onedrive_base / 'Portfolio Management Team' / '2 - Monthly AUM and Performance' / 'DEV'
input_path = onedrive_base / process_base
output_path = onedrive_base / process_base

# In order, outlining the value to search for in "Column A", then the name of the column to get the value for, then the name of the new column when we flatten out the file
mapping_rules = (
    ('Total', 'Starting Adjusted Value (No Div, USD)', 'Starting Value (12/31/2024)'),
    ('Total', 'Adjusted Total Return (No Div, USD)', 'Total Return'),
    ('Total', 'Adjusted Value (No Div, USD)', 'Ending Value (11/30/2025)'),
    ('Total', 'Adjusted TWR (Current Month, No Div, USD)', 'Nov 2025 TWR'),
    ('Total', 'Adjusted TWR (YTD, No Div, USD)', 'YTD TWR'),
    ('Total', '1 Yr. Adjusted TWR (No Div, USD)', '1Y TWR'),
    ('Total', '3 Yr. Annualized Adjusted TWR (No Div, USD)', '3Y TWR'),
    ('Equities', 'Adjusted TWR (YTD, No Div, USD)', 'EQ - YTD TWR'),
    ('Equities', '1 Yr. Adjusted TWR (No Div, USD)', 'EQ - 1Y TWR'),
    ('Equities', '3 Yr. Annualized Adjusted TWR (No Div, USD)', 'EQ - 3Y TWR'),
    ('Fixed Income', 'Adjusted TWR (YTD, No Div, USD)', 'FI - YTD TWR'),
    ('Fixed Income', '1 Yr. Adjusted TWR (No Div, USD)', 'FI - 1Y TWR'),
    ('Fixed Income', '3 Yr. Annualized Adjusted TWR (No Div, USD)', 'FI - 3Y TWR'),
    ('Cash & Cash Equivalents', 'Adjusted Value (No Div, USD)', 'Total - Cash'),
    ('Fixed Income', 'Adjusted Value (No Div, USD)', 'Total - FI'),
    ('Equities', 'Adjusted Value (No Div, USD)', 'Total - Equities'),
    ('Private Markets', 'Adjusted Value (No Div, USD)', 'Total - Private Markets'),
)

In [ ]:
# Set directory
os.chdir(input_path)

# Brining in all files meeting these requirements 
excel_files = [f for f in os.listdir(input_path) 
               if f.startswith("Perf ")
               and f.endswith(('.xlsx', '.xls'))
               #and f in included_organizations_list
              ]

# Just want to get a count of all excel files as another QC step
all_excel_files = [f for f in os.listdir(input_path) 
                   if f.endswith(('.xlsx', '.xls'))]
total_excel_count = len(all_excel_files)

# Validation seen below
print('Number of files meeting criteria above found:', len(excel_files))
print(f'Total Excel files in directory: {total_excel_count}')
print('List of files to be imported:', excel_files)

Number of files meeting criteria above found: 23
Total Excel files in directory: 27
List of files to be imported: ['Perf Algonquin.xlsx', 'Perf B&J.xlsx', 'Perf Bainco.xlsx', 'Perf Bloomfield Hills.xlsx', 'Perf Buckley.xlsx', 'Perf Chicago EFC.xlsx', 'Perf Chicago Ty Parrish.xlsx', 'Perf Cleveland.xlsx', 'Perf EMM.xlsx', 'Perf Greenbrae.xlsx', 'Perf Individual Advisors.xlsx', 'Perf Karsten.xlsx', 'Perf Kurosko.xlsx', 'Perf Los Altos.xlsx', 'Perf Moore.xlsx', 'Perf Nardi.xlsx', 'Perf Norfolk 2.xlsx', 'Perf Norfolk.xlsx', 'Perf Park.xlsx', 'Perf PCA.xlsx', 'Perf Rowley.xlsx', 'Perf Shier Team.xlsx', 'Perf Worldview.xlsx']


In [4]:
all_results = []

for file in excel_files:
    #print(f"\nProcessing: {file}")
    
    # Load Excel file
    file_path = os.path.join(input_path, file)
    data = pd.read_excel(file_path)

    results = {}
    for row_value, input_name, new_column_name in mapping_rules:
        # Find rows where the first column matches the row_value
        mask = data.iloc[:, 0] == row_value  # First column (A)
        
        if mask.any() and input_name in data.columns:
            # Get the first matching value from the specified column
            value = data.loc[mask, input_name].iloc[0]
            
            results[new_column_name] = value
        else:
            results[new_column_name] = None  
    
    key_name = os.path.splitext(file)[0]
    last_word = key_name.split(' ', 1)[-1] if ' ' in key_name else key_name
    results['Organization'] = last_word
    
    # Add to results list
    all_results.append(results)

# Create a new DataFrame with the extracted values
output_data = pd.DataFrame(all_results)

C:\Users\azona\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\azona\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\azona\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\azona\AppData\Local\Packages\Pytho

In [ ]:
# Creating "net cash flow" column
output_data['Net Cash Flow'] = output_data['Ending Value (11/30/2025)'] - output_data['Total Return'] - output_data['Starting Value (12/31/2024)']

In [6]:
# Organizing the sequence of data
columns_sequence = ['Organization', 'Starting Value (12/31/2024)', 'Net Cash Flow' , 'Total Return', 'Ending Value (11/30/2025)', 'Nov 2025 TWR', 'YTD TWR', '1Y TWR', '3Y TWR', 'EQ - YTD TWR', 
        'EQ - 1Y TWR', 'EQ - 3Y TWR', 'FI - YTD TWR', 'FI - 1Y TWR', 'FI - 3Y TWR', 'Total - Cash', 'Total - FI', 'Total - Equities', 'Total - Private Markets']

output_data = output_data[columns_sequence]

In [ ]:
# Outlining who belongs in what "Organization"
org_mapping = {
    'Chicago': ['Ahamed', 'Kuehl' , 'Nardi' , 'Rowley' , 'Schoneman' , 'Silverman'],
    'New York': ['Dearing', 'Farrell', 'Jumper', 'Kurosko', 'Lebenthal' , 'Nadel' , 'Ruttenberg' , 'Tchetchoulina' , 'Weissman'],
    'NYC EFC': ['Buckley' , 'Carrington' , "D'Elia" , 'Moore' , 'Park'],
    'OC EFC': ['Jimenez' , 'Shier'],
    'OC WM': ['Larr' , 'Nielander']
}

# Identifying which columns will be summed 
numeric_columns_for_sum = [
    'Starting Value (12/31/2024)', 
    'Net Cash Flow',
    'Total Return', 
    'Ending Value (11/30/2025)',
    'Total - Cash',
    'Total - FI',
    'Total - Equities',
    'Total - Private Markets'
]

In [ ]:
# Mapping the flags to the spreadsheets
def assign_org_flag(org_name):
    for flag, org_list in org_mapping.items():
        if org_name in org_list:
            return flag
    return None  

# creating the column
output_data['org_flag'] = output_data['Organization'].apply(assign_org_flag)

In [ ]:
grouped_rows = []
for flag in org_mapping.keys():
    # Filter rows with this flag
    flag_data = output_data[output_data['org_flag'] == flag]
    
    if not flag_data.empty:
        # Sum numeric columns
        summary = flag_data[numeric_columns_for_sum].sum()
        
        # Create sum row
        summary_row = summary.to_dict()
        summary_row['Organization'] = flag
        summary_row['org_flag'] = flag
        
        grouped_rows.append(summary_row)

# Make grouped rows a dataframe
grouped_df = pd.DataFrame(grouped_rows)

# Combine original data with grouped dataframe
output_data = pd.concat([output_data, grouped_df], ignore_index=True)

In [17]:
output_data.to_excel('combined_output.xlsx', index=False)